In [ ]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
plt.rcParams['figure.figsize'] = [9, 8]

import cv2
import numpy as np
import pandas as pd
pd.set_option('display.width', 74)
pd.set_option('display.max_columns', 15)
pd.set_option('display.max_rows', 20)

In [ ]:
import os
import datetime
import itertools

from sklearn.metrics import confusion_matrix

DATADIR = "data/keras"
NUM_CHARACTERS = len(os.listdir(os.path.join(DATADIR, "train", "images")))

In [ ]:
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, Input, Flatten
from keras.layers.normalization import BatchNormalization
from keras.models import Model
from keras.optimizers import Adam

# Image processing
IMAGE_ROW_SIZE = 584
IMAGE_COLUMN_SIZE = 480

def init_model(target_num=4, dropout_ratio=0.5, learning_rate=0.0001):
    input_shape = (IMAGE_ROW_SIZE, IMAGE_COLUMN_SIZE, 3)

    # Fine-tune prediction layer
    pretrained_model = VGG16(include_top=False, weights='imagenet',
                             input_shape=input_shape)
    for layer in pretrained_model.layers:
        layer.trainable = False

    output_tensor = pretrained_model.output
    output_tensor = Flatten()(output_tensor)
    output_tensor = Dense(128, activation='relu')(output_tensor)
    # output_tensor = Dense(128, activation='relu')(output_tensor)
    output_tensor = Dense(target_num, activation="softmax", name="predictions")(output_tensor)

    # Define and compile the model
    model = Model(inputs=pretrained_model.input, outputs=output_tensor)
    model.compile(optimizer=Adam(lr=learning_rate),
                  loss="categorical_crossentropy",
                  metrics=["accuracy"])

    return model

model = init_model(target_num=NUM_CHARACTERS)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

def make_generator(folder="train",
                   data_gen_args={"fill_mode": "constant",
                                  "cval": 0,
                                  "width_shift_range": 0.05,
                                  "height_shift_range": 0.05,
                                  "zoom_range": 0.1,
                                  "horizontal_flip": True,
                                  "rescale": 1.0 / 255},
                   data_flow_args={"seed": 1,
                                   "batch_size": 32}):

    image_datagen = ImageDataGenerator(**data_gen_args)

    image_generator = image_datagen.flow_from_directory(
        directory=os.path.join(DATADIR, folder, "images"),
        target_size=(IMAGE_ROW_SIZE, IMAGE_COLUMN_SIZE),
        color_mode='rgb',
        **data_flow_args)

    return image_generator

def steps_per_epoch(folder="train", batch_size=32):
    image_directory = os.path.join(DATADIR, folder, "images")
    char_name = os.listdir(image_directory)[0]
    char_directory = os.path.join(image_directory, char_name)
    data_size = len(os.listdir(char_directory))
    return data_size // batch_size

In [ ]:
model.summary()

In [ ]:
model.fit_generator(generator=make_generator("train"),
                    steps_per_epoch=steps_per_epoch("train"),
                    epochs=10,
                    validation_data=make_generator("valid"),
                    validation_steps=steps_per_epoch("valid"))

In [ ]:
model.save(os.path.join(DATADIR, datetime.datetime.today().strftime('%Y-%m-%d') + "-short-epochs-1-hidden-layer.h5"))

In [ ]:
def make_test_generator(folder="test",
                        data_flow_args={"seed": 1,
                                        "batch_size": 32}):

    return ImageDataGenerator().flow_from_directory(
        directory=os.path.join(DATADIR, folder, "images"),
        target_size=(IMAGE_ROW_SIZE, IMAGE_COLUMN_SIZE),
        color_mode='rgb',
        **data_flow_args)

In [ ]:
test_batches = make_test_generator()
class_dict = test_batches.class_indices
index_dict = {i: c for c, i in class_dict.items()}

In [ ]:
def plot_confusion_matrix(matrix, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        matrix = matrix.astype('float') / matrix.sum(axis=1)[:, np.newaxis]

    plt.imshow(matrix, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = matrix.max() / 2.
    for i, j in itertools.product(range(matrix.shape[0]), range(matrix.shape[1])):
        plt.text(j, i, format(matrix[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if matrix[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
images, vectors = next(test_batches)
true_indices = vectors.argmax(1)
predictions = model.predict_on_batch(images)
prediction_indices = predictions.argmax(1)
class_names = [c for i, c in sorted(index_dict.items())]

cnf_matrix = confusion_matrix(true_indices, prediction_indices)
plot_confusion_matrix(cnf_matrix, classes=class_names, normalize=True,
                      title='Normalized confusion matrix')